In [1]:
from ephysvibe.structures.neuron_data import NeuronData
from ephysvibe.trials.spikes import firing_rate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
def get_aprox_pos(x):
    xabs = abs(x)
    xsign = np.sign(x)
    if xabs<3.5:
        x = 0
    elif xabs>=3.5 and xabs<8.5:
        x = xsign*7
    elif xabs>=8.5:
        x = xsign*10
    return x

In [3]:
def get_v_resp_loc(neu,params,rf_loc=None):
    # Check rf of the neurons
    if rf_loc is not None:
        neu.check_fr_loc(rf_loc)
    nid = neu.get_neuron_id()
    # Define masks
    b1_mask = neu.block == 1
    b2_mask = neu.block == 2
    in_mask = neu.pos_code == 1
    # check b2 
    if np.sum(b2_mask)==0:
        results = {'nid':nid,
            'v_resp_out':np.nan,
            'x_pos_b2':np.nan,
            'y_pos_b2':np.nan,
            'x_pos_b1':np.nan,
            'y_pos_b1':np.nan,
            'code':np.nan,
            'op_code':np.nan
            }
        return results    
    # get the screen position of sample in b1 in contralateral trials
    u_pos, u_count = np.unique(neu.position[np.logical_and(b1_mask,in_mask)], axis=0,return_counts =True)
    imax = np.argmax(u_count)
    x_pos_b1, y_pos_b1 = u_pos[imax][0][0], u_pos[imax][0][1]
    # Concatenate and get unique position and code during b2 trials
    pos_and_code = np.concatenate([neu.position[b2_mask][:,0],neu.pos_code[b2_mask].reshape(-1,1)],axis=1)
    u_pos = np.unique(pos_and_code, axis=0)
    # Find the closest screen position to b1 in b2
    diff = abs(abs(u_pos[:,:2])-abs(np.array([x_pos_b1, y_pos_b1])) )
    idx = np.argmin(np.sum(diff,axis=1))
    x_pos_b2 = abs(u_pos[idx,0]) * np.sign(x_pos_b1)
    y_pos_b2 = abs(u_pos[idx,1]) * np.sign(y_pos_b1)
    idx_in = np.logical_and(u_pos[:,0]==x_pos_b2, u_pos[:,1]==y_pos_b2)
    # if np.sum(idx_in)==0:
    #     x_pos = get_aprox_pos(x_pos)
    #     y_pos = get_aprox_pos(y_pos)
    #     idx_in = np.logical_and(u_pos[:,0]==x_pos, u_pos[:,1]==y_pos)
    code_in = str(int(u_pos[idx_in][0][2]))
    idx_out = np.logical_and(u_pos[:,0]==-x_pos_b2, u_pos[:,1]==-y_pos_b2)
    code_out = str(int(u_pos[idx_out][0][2]))
    # Align spikes to target onset and get trials by code
    align_sp, alig_mask = neu.align_on(select_block=2,event='target_on',time_before=200,error_type=0)
    pos_code = neu.pos_code[alig_mask]
    sp_pos = {}
    for code in np.unique(pos_code):
        code_mask = pos_code == code
        sp_pos[str(int(code))] = align_sp[code_mask]
    # Compute t-test comparing baseline with target presentation (in vs opposite loc)
    sp = sp_pos[code_in] 
    sp_op = sp_pos[code_out]
    # Check fr
    fr = np.nanmean(sp[:,:1000])*1000
    if fr<1:
        results = {'nid':nid,
            'v_resp_out':np.nan,
            'x_pos_b2':np.nan,
            'y_pos_b2':np.nan,
            'x_pos_b1':np.nan,
            'y_pos_b1':np.nan,
            'code':np.nan,
            'op_code':np.nan
            }
        return results        
    p = stats.ttest_rel(np.mean(sp[:,:200],axis=1), np.mean(sp[:,200:400],axis=1))
    p = p[1]<0.05
    p_op = stats.ttest_rel(np.mean(sp_op[:,:200],axis=1), np.mean(sp_op[:,200:400],axis=1))
    p_op = p_op[1]<0.05
    v_resp_out = True
    # check loc of rf in b1
    if p ==False and p_op == False: 
        v_resp_out = False
    elif np.all(neu.rf_loc[b1_mask] == neu.pos_code[b1_mask]):
        if p and p_op == False: v_resp_out = False
    else:
        if p_op and p == False: v_resp_out = False
    results = {'nid':nid,
        'v_resp_out':v_resp_out,
        'x_pos_b2':x_pos_b2,
        'y_pos_b2':y_pos_b2,
        'x_pos_b1':x_pos_b1,
        'y_pos_b1':y_pos_b1,
        'code':code_in,
        'op_code':code_out
        }
    return results

In [4]:
oposite_codes= {
    '127': '123',
    '126': '122',
    '125': '121',
    '124': '120',
    '123': '127',
    '122': '126',
    '121': '125',
    '120': '124',
}

In [5]:
def read_and_compute(path, params, rf_loc=None):
    neu = NeuronData.from_python_hdf5(path)
    neu = get_v_resp_loc(params=params, neu=neu, rf_loc=rf_loc)
    return neu

In [6]:
import pandas as pd
import glob
from joblib import Parallel, delayed
from tqdm import tqdm

In [ ]:
processing={}
paths={'input_files':"//envau_cifs.intlocal.univ-amu.fr/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/new_structure/session_struct/pfc/neurons/",
       'input_rf_loc':"//envau_cifs.intlocal.univ-amu.fr/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/activation_index/rf_loc_df_pfc.csv"}
params = []
for idict in processing:
    params.append(processing[idict])
neu_path = paths["input_files"] + "*neu.h5"
path_list = glob.glob(neu_path)
path_list = np.array(path_list)[np.arange(0,100)]
rf_loc = None
if paths["input_rf_loc"] is not None:
    rf_loc = pd.read_csv(paths["input_rf_loc"])

res = Parallel(n_jobs=-1)(
    delayed(read_and_compute)(path, params, rf_loc) for path in tqdm(path_list)
)

df = pd.DataFrame(res)

 56%|█████▌    | 56/100 [09:18<09:13, 12.59s/it]

In [8]:
df[df['v_resp_out']==True]

,nid,v_resp_out,x_pos_b2,y_pos_b2,x_pos_b1,y_pos_b1,code,op_code
18,2022-11-22_10-59-03_Riesling_pfc_e1_r1_mua5,True,8.4,-8.4,5.0,-5.0,120,124
20,2022-11-22_10-59-03_Riesling_pfc_e1_r1_mua14,True,8.4,-8.4,5.0,-5.0,120,124
35,2023-02-09_10-42-54_Riesling_pfc_e1_r1_mua32,True,4.9,-4.9,5.0,-5.0,120,124
62,2022-12-06_11-12-04_Riesling_pfc_e1_r1_mua12,True,7.0,-7.0,3.0,-5.0,120,124
70,2022-12-14_10-50-49_Riesling_pfc_e1_r1_mua7,True,3.5,-3.5,4.0,-4.0,120,124
74,2022-12-14_10-50-49_Riesling_pfc_e1_r1_good11,True,3.5,-3.5,4.0,-4.0,120,124
77,2022-12-14_10-50-49_Riesling_pfc_e1_r1_good19,True,3.5,-3.5,4.0,-4.0,120,124
78,2022-12-14_10-50-49_Riesling_pfc_e1_r1_mua27,True,3.5,-3.5,4.0,-4.0,120,124
79,2022-12-14_10-50-49_Riesling_pfc_e1_r1_mua32,True,3.5,-3.5,4.0,-4.0,120,124
81,2022-12-16_10-30-59_Riesling_pfc_e1_r1_mua6,True,3.5,-3.5,4.0,-4.0,120,124


In [9]:
df[df['v_resp_out']==False]

,nid,v_resp_out,x_pos_b2,y_pos_b2,x_pos_b1,y_pos_b1,code,op_code
0,2022-12-01_10-23-18_Riesling_pfc_e1_r1_mua1,False,10.0,0.0,10.0,0.0,127,123
4,2022-12-01_10-23-18_Riesling_pfc_e1_r1_mua15,False,10.0,0.0,10.0,0.0,127,123
5,2022-12-01_10-23-18_Riesling_pfc_e1_r1_mua19,False,10.0,0.0,10.0,0.0,127,123
6,2022-12-01_10-23-18_Riesling_pfc_e1_r1_good8,False,10.0,0.0,10.0,0.0,127,123
8,2022-11-28_10-23-27_Riesling_pfc_e1_r1_mua2,False,4.9,-4.9,5.0,-5.0,120,124
9,2022-11-28_10-23-27_Riesling_pfc_e1_r1_good3,False,4.9,-4.9,5.0,-5.0,120,124
10,2022-11-28_10-23-27_Riesling_pfc_e1_r1_mua7,False,4.9,-4.9,5.0,-5.0,120,124
11,2022-11-28_10-23-27_Riesling_pfc_e1_r1_good11,False,4.9,-4.9,5.0,-5.0,120,124
12,2022-11-28_10-23-27_Riesling_pfc_e1_r1_mua10,False,4.9,-4.9,5.0,-5.0,120,124
13,2022-11-28_10-23-27_Riesling_pfc_e1_r1_mua14,False,4.9,-4.9,5.0,-5.0,120,124


In [21]:
path_list = glob.glob(neu_path)
for i in np.arange(450,500,5):
    read_and_compute(path_list[i], params, rf_loc)

In [17]:
i

np.int64(490)

In [12]:
path_list = glob.glob(neu_path)

In [19]:
read_and_compute(path_list[490], params, rf_loc)

ValueError: attempt to get argmin of an empty sequence

In [27]:
df[np.logical_and(~np.isnan(df['x_pos']),np.logical_and(df['v_resp_out']!=True,df['y_pos']!=False))]

,nid,v_resp_out,x_pos,y_pos,code,op_code
8,2022-11-28_10-23-27_Riesling_pfc_e1_r1_mua2,NaN,5.0,-5.0,NaN,NaN
9,2022-11-28_10-23-27_Riesling_pfc_e1_r1_good3,NaN,5.0,-5.0,NaN,NaN
10,2022-11-28_10-23-27_Riesling_pfc_e1_r1_mua7,NaN,5.0,-5.0,NaN,NaN
11,2022-11-28_10-23-27_Riesling_pfc_e1_r1_good11,NaN,5.0,-5.0,NaN,NaN
12,2022-11-28_10-23-27_Riesling_pfc_e1_r1_mua10,NaN,5.0,-5.0,NaN,NaN
...,...,...,...,...,...,...
95,2022-12-13_11-11-42_Riesling_pfc_e1_r1_mua25,NaN,4.0,-4.0,NaN,NaN
96,2022-12-13_11-11-42_Riesling_pfc_e1_r1_mua28,NaN,4.0,-4.0,NaN,NaN
97,2022-12-13_11-11-42_Riesling_pfc_e1_r1_mua31,NaN,4.0,-4.0,NaN,NaN
98,2023-01-18_10-22-38_Riesling_pfc_e1_r1_mua1,NaN,7.0,-5.0,NaN,NaN
